# Exercise 3: Perturbing temperature and precipitation

In this excercise you will explore how different perturbations to temperature and precipitation propagate through both MetSim and SUMMA and how this changes the resulting hydrology.
As before, we will import everything we need. You are also provided with an initial MetSim configuration which is the same as the one from the previous exercise.  We have also included the same helper functions from the previous exercise. Run these three cells to get started.


In [ ]:
# modules 
import os
import pysumma as ps
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from metsim import MetSim

In [ ]:
def metsim_to_summa(ms, base_dataset='./data/reynolds/forcing_sheltered.nc'):
    """
    Convert a metsim object's output to summa compatible input
    This simply adds the `data_step` and `hruId` variables from
    the given `base_dataset`. This appends to the metsim dataset.
    
    Parameters
    ----------
    ms: MetSim
        A MetSim object which has had the `run` method called
    base_dataset: str
        The path to a dataset which is used to populate the
        `data_step` and `hruId` variables
        
    Returns
    -------
    The update `xarray.Dataset` from the MetSim run
    """
    base_dataset = xr.open_dataset(base_dataset)
    with ms.open_output() as ds:
        ds['data_step'] = base_dataset['data_step']
        ds['hruId'] = base_dataset['hruId']
        out_ds = ds.load()
    out_prefix = ms.params["out_prefix"]
    out_suffix = ms.get_nc_output_suffix(out_ds["time"].to_series())
    out_filename = f'{out_prefix}_{out_suffix}.nc'
    out_dirname = os.path.abspath(ms.params['out_dir'])
    out_ds.to_netcdf(f'{out_dirname}/{out_filename}')
    return out_ds

    
def create_file_manager(ms, fm, new_suffix):
    """
    Creates a new file manager that points to a forcing
    file from a metsim run.
    
    Parameters
    ----------
    ms: MetSim
        A MetSim object which has had the `run` method called.
    fm: FileManager
        A FileManager to use as a template
    new_suffix: str
        The new suffix for the new file manager
        
    Returns
    -------
    The path to the new file manager
    """
    out_prefix = ms.params["out_prefix"]
    with ms.open_output() as ds:
        out_suffix = ms.get_nc_output_suffix(ds["time"].to_series())
    out_filename = f'{out_prefix}_{out_suffix}.nc'
    out_dirname = os.path.abspath(ms.params['out_dir'])
   
    new_force_file_list = f'summa_zForcingFileList_{new_suffix}.txt'
    new_file_manager = f'summa_fileManager_{new_suffix}.txt'
    with open(f'{fm["settingsPath"].value}/{new_force_file_list}', 'w') as f:
        f.write(f"'{out_filename}'")
    fm['forcingListFile'].value = new_force_file_list
    fm.file_name = new_file_manager
    fm.write()
    return f'./{fm.original_path}/{fm.file_name}'

In [ ]:
config = {
    # Input files
    "domain": './data/reynolds/forcing_daily.nc',
    "forcing": './data/reynolds/forcing_daily.nc',
    "state": './data/reynolds/forcing_daily.nc',
    # Output location/naming
    "out_dir": './data/reynolds/',
    "out_prefix": 'forcing_metsim_uniform',
    # Run configuration/parameters
    "start": "2005/06/01",
    "stop": "2006/10/01",
    "time_step": 60,
    "period_ending": True,
    # Set up spatial chunking
    "chunks": {'hru': 1},
    # Set up input variable mapping
    "forcing_vars": {"Tmin": "t_min", "Tmax": "t_max", "prcp": "prec", "wind": "wind",},
    "state_vars": {"Tmin": "t_min", "Tmax": "t_max", "prcp": "prec", "wind": "wind",},
    "domain_vars": {"lon": "lon", "lat": "lat", "elev": "elev", "mask": "mask",},
    # Set up output specifications
    "out_vars": {
        'temp': {'out_name': 'airtemp', 'units': 'K'}, 'prec': {'out_name': 'pptrate', 'units': 'mm s-1'},
        'air_pressure': {'out_name': 'airpres', 'units': 'Pa'}, 'shortwave': {'out_name': 'SWRadAtm'},
        'longwave': {'out_name': 'LWRadAtm'}, 'spec_humid': {'out_name': 'spechum' }, 'wind': {'out_name': 'windspd' } },
}

# Defining a daily perturbation function

Your first task is to define your own perturbation functions. We have provided you a scaffold below which you can use to modify `Tmin`, `Tmax` and, `prcp`. While you are free to choose any perturbations you like, they should be physically plausible. You may also write a more complex function if you desire. Justify the choices you make here.

In [ ]:
def perturb_daily_vals(ds, dT, dP, out_path):
    """
    Modify the temperature and precipitation inputs
    to MetSim. 
    
    Parameters
    ----------
    ds: xr.Dataset
        A forcing dataset for MetSim
    dT: float
        The numeric perturbation for temperature
    dP: float
        The numeric perturbation for precipitation
    out_path: string
        Where to write the new data out
        
    Returns
    -------
    The perturbed dataset (which is also written out to disk)
    """
    # We don't want to overwrite the original
    # data so that we can iterate over it for
    # producing multiple perturbations
    ds = ds.copy()
    
    # Todo: 
    # insert your own perturbations here
    
    # Don't forget to write out the data!
    ds.to_netcdf(out_path)
    return ds

# Running MetSim and preparing to run SUMMA

As in the last exercise, now we will run MetSim for a number of these perturbations and prepare those MetSim outputs to run SUMMA.
We have provided some of the scaffolding here to do this, including a suggested set of perturbation values to be fed into your
perturbation functions. Feel free to change or add to these as you see fit. We also have provided a short checklist for the inner loop
to help you get started.

As before, once you have run all of these though MetSim, make sure to plot the resulting temperature and precipitation timeseries.
Comment on how your perturbations have affected these timeseries. You may optionally want to explore some of the other MetSim outputs as well.

In [ ]:
temp_perturbation = [-1.0, 0.0, 1.0]
prec_perturbation = [-5.0, 0.0, 5.0]
new_file_managers = []
run_names = []
metsim_outputs = []
daily_ds = xr.open_dataset('./data/reynolds/forcing_daily.nc')
for dT in temp_perturbation:
    for dP in prec_perturbation:
        print(f'Running dT={dT}, dP={dP} now!')
        out_name = f'dT={dT}_dP={dP}'
        run_names.append(out_name)
        out_daily_path = f'./data/reynolds/forcing_daily_{out_name}.nc'
        perturb_daily_vals(daily_ds, dT, dP, out_daily_path)
        # Todo:
        #  1. Update `out_prefix` in the metsim configuration with `out_name`
        #  2. Update `forcing`, `state`, and `domain` entries in the config 
        #     using the `out_daily_path`
        #  3. Instantiate and run metsim
        #  4. Convert metsim output to summa input, and append to `metsim_outputs`
        #  5. Create a new file manager for summa simulations, and append to `new_file_managers`

# Running SUMMA

Now that you've got perturbed datasets, run them through SUMMA similarly to how we did before. We have provided you an additional snippet which makes it easy to slice through either temperature or precipitation perturbations. To analyze how your perturbations have changed the simulations conduct a simple sensitivity analysis to see whether your temperature or precipitation perturbations have had a larger impact on the simulated SWE. Note whether any particular perturbation had a large effect on the SWE.

In particular, which perturbations have the largest impact on the peak SWE? Which have the largest impact on the snow season duration (amount of time with snow on the ground).

In [ ]:
assert len(new_file_managers) == 9, ('The previous step is still not complete! Please go back',
                                     ' and fill lin the loop from the previous code block to continue')

ens_config = {name: {'file_manager': nfm}
              for name, nfm in zip(run_names, new_file_managers)}

summa_executable = 'summa.exe'
file_manager = './settings/reynolds/summa_fileManager.txt'
pert_ens = ps.Ensemble(summa_executable, ens_config, num_workers=3)

pert_ens.run('local')
summary = pert_ens.summary()
print(summary)

In [ ]:
ds_list = [s.output.load().isel(hru=0, gru=0) for s in pert_ens.simulations.values()]
pert_ds = xr.concat(ds_list, dim='run')
new_dims = [(dT, dP) for dT in temp_perturbation for dP in prec_perturbation]
pert_ds['run'] = pd.MultiIndex.from_tuples(new_dims, names=('dT', 'dP'))
pert_ds = pert_ds.unstack()

In [ ]:
# Todo:
# 1. plot the SWE timeseries for each of the perturbations
# 2. Compute and plot the peak swe and length of snow season for each of 
#    the perturbations
# 3. Compute the sensitivity of snow season and peak swe as a function of 
#    the temperature perturbations
# 4. Compute the sensitivity of snow season and peak swe as a function of 
#    the precipitation perturbations
# 5. Comment on which perturbations had larger impacts on these quantities. 
#    Is this in line with your expectations? Do you expect this is true everywhere?